In [113]:
#from datetime import datetime
#import pytz
#import json
#import urllib
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, LinearAxis, Range1d, Band, BoxSelectTool
from bokeh.models import HoverTool, TapTool, DatetimeTickFormatter, DateFormatter, LabelSet
from bokeh.tile_providers import get_provider, Vendors
from scipy.spatial.distance import cdist
#from bokeh.models.renderers import GlyphRenderer
#from bokeh.models.widgets import DataTable, TableColumn, Slider, Dropdown
from bokeh.layouts import layout, column, row, widgetbox
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [2]:
base_url = 'http://meteo145.uibk.ac.at/'
station = 'innsbruck'
time = str(3)

url = base_url + station + '/' + time

In [3]:
df = pd.read_json(url)
df['time'] = [datetime(1970, 1, 1) + timedelta(milliseconds=ds) for ds in df['datumsec']]
df = df.set_index('time')
df = df.drop(columns='datumsec')
df.head()

,rr,dd,tp,p,tl,so,ff
time,,,,,,,
2019-10-21 10:00:00,0.0,343,9.9,949.2,21.5,10.000000,2.8
2019-10-21 10:10:00,0.0,264,10.3,949.3,21.7,5.083333,1.2
2019-10-21 10:20:00,0.0,187,8.9,949.2,22.4,9.933333,0.8
2019-10-21 10:30:00,0.0,146,9.0,949.1,23.0,10.000000,4.3
2019-10-21 10:40:00,0.0,172,8.8,949.1,23.5,10.000000,1.6


In [4]:
for column in df.columns:
    df[df[column] == -99.9] = np.nan


In [5]:
def merc(lat, lon):
    """Convert latitude and longitude into mercator's x and y position
    Parameters
    ----------
    lon : longitude
    lat : latitude
    """
    r_major = 6378137.000
    x = r_major * np.radians(lon)
    scale = x/lon
    y = 180.0/np.pi * np.log(np.tan(np.pi/4.0 + 
        lat * (np.pi/180.0)/2.0)) * scale
    return (x, y)

In [6]:
stations = pd.DataFrame({'lat':[47.26], 'lon':[11.45]}, index=['innsbruck'])
[stations['x'], stations['y']]= merc(stations['lat'],stations['lon'])
stations

,lat,lon,x,y
innsbruck,47.26,11.45,1.274608e+06,5.984616e+06


In [7]:
tile_provider = get_provider(Vendors.CARTODBPOSITRON)
map_tools = 'box_zoom,pan,save,hover,reset,tap,wheel_zoom'
map_plot = figure(x_range=(1108137, 1417582), y_range=(5895123 , 6088551), plot_width=600, plot_height=350,
           x_axis_type="mercator", y_axis_type="mercator", tools=map_tools)#, sizing_mode="scale_both")
map_plot.add_tile(tile_provider)
map_plot.circle(x="x", y="y", size=15, fill_color="blue", fill_alpha=0.4, source=stations);
hover_map = map_plot.select(dict(type=HoverTool))
hover_map.tooltips = [("Stationname", "@index")]
hover_map.mode = 'mouse'
show(map_plot)

In [8]:
df['z'] = 0 #adding zero column to display band for ssd

In [9]:
def get_width():
    mindate = min(df.index)
    maxdate = max(df.index)
    return 0.8 * (maxdate-mindate).total_seconds()*1000 / len(df.index)
get_width()

478888.8888888889

In [116]:
fwidth = 900
fhgt = 400
font_size_label = "20pt"
font_size_ticker = "15pt"

In [125]:
#source = ColumnDataSource(df)

p1_tools = 'box_zoom,pan,save,hover,reset,xwheel_zoom'
p1 = figure(width = fwidth, height = fhgt, x_axis_type="datetime", tools=p1_tools);

p1.xaxis.axis_label_text_font_size = font_size_label
p1.yaxis.axis_label_text_font_size = font_size_label
p1.xaxis.major_label_text_font_size = font_size_ticker
p1.yaxis.major_label_text_font_size = font_size_ticker
p1.yaxis.major_label_text_font_size = font_size_ticker

p1.xaxis.formatter=DatetimeTickFormatter(
        hours=['%H:%M'],
        days=["%b %d %Y"],
    )

# sunshine duration
p1.extra_y_ranges = {'ssd': Range1d(start=0, end=10)}
p1.add_layout(LinearAxis(y_range_name='ssd', axis_label='Sunshine duration ()'), 'right')
p1.vbar(top='so', x='time', source=df, width=get_width(), fill_color='yellow', 
        line_alpha=0, line_width=0, fill_alpha=0.5, y_range_name='ssd', legend = 'Sunshine duration')
p1.yaxis[1].axis_label = 'Sunshine duration (min)'
p1.yaxis[1].axis_label_text_font_size = font_size_label
p1.yaxis[1].major_label_text_font_size = font_size_ticker

# temperature
h_line = p1.line(x='time', y='tl', source=df, line_width=4, color='red', legend='Temperature');
p1.yaxis[0].axis_label = 'Temperature (°C)'

# dew point
p1.y_range=Range1d(df['tp'].min()-2, df['tl'].max()+2)
p1.line(x='time', y='tp', source=df, line_width=4, color='green', legend = 'Taupunkt')

# hover
hover_p1 = p1.select(dict(type=HoverTool))

hover_p1.tooltips = [("Timestamp", "@time{%Y-%m-%d %H:%M}"), 
                     ('Air temperature', '@tl'), 
                     ('Dewpoint', '@tp'), 
                     ('Sunshine duration', '@so')]#
hover_p1.formatters = { "time": "datetime"}
hover_p1.mode = 'vline'
hover_p1.renderers =[h_line] #### to fix if missing value

# legend
p1.legend.location = "top_left"
p1.legend.click_policy="hide"

#p1.add_tools(BoxSelectTool(dimensions="width"))

show(p1)

In [98]:
ffcol = 'red'
ddcol = 'black'
pcol = 'blue'

In [129]:
p2_tools = 'box_zoom,pan,save,hover,reset,wheel_zoom'
p2 = figure(width = fwidth, height = fhgt,x_axis_type="datetime", tools=p2_tools);
p2.xaxis.formatter=DatetimeTickFormatter(
        hours=['%H:%M'],
        days=["%b %d %Y"],
    )

p2.xaxis.axis_label_text_font_size = font_size_label
p2.yaxis.axis_label_text_font_size = font_size_label
p2.xaxis.major_label_text_font_size = font_size_ticker
p2.yaxis.major_label_text_font_size = font_size_ticker
p2.yaxis.major_label_text_font_size = font_size_ticker

# pressure
h_line = p2.line(x='time', y='p', source=df, line_width=1.5, color='blue')
p2.y_range=Range1d(df['p'].min()-10, df['p'].max()+10)
p2.yaxis[0].axis_label = 'Pressure (hPa)'

# wind
#p2.extra_y_ranges = {'winddir': Range1d(start=0, end=360)}
p2.extra_y_ranges = {"winddir": Range1d(start=0, end=360), 
                     "windspd": Range1d(start=0, end=df['ff'].max()+df['ff'].max()*0.1)}
p2.add_layout(LinearAxis(y_range_name='winddir'), 'right')
p2.add_layout(LinearAxis(y_range_name="windspd"), 'right')
p2.circle(x='time', y='dd', source=df, line_width=1.5, color='black', y_range_name='winddir')
p2.line(x='time', y='ff', source=df, line_width=1.5, color='red', y_range_name='windspd')
#p2.patch(x='time', y = 'ff', source = df, color = "black",alpha=0.5, line_width=2)
hover_p2 = p2.select(dict(type=HoverTool))

hover_p2.tooltips = [("Timestamp", "@time{%Y-%m-%d %H:%M}"), 
                     ('Pressure', '@p'), 
                     ('Winddirection', '@dd'),
                     ('Windspeed', '@ff')]
hover_p2.formatters = { "time": "datetime"}
hover_p2.mode = 'vline'
hover_p2.renderers =[h_line] #### to fix if missing value

show(p2)

p1.yaxis[0].major_label_text_color = tlcol  



In [45]:
from bokeh.themes import theme


In [104]:
p2

Figure(id='3254', ...)

In [ ]:
p1.add_tools(BoxSelectTool(dimensions="width"))